In [1]:
from psd_tools import PSDImage
from PIL import Image
import cv2
from lib.image_processing import transform
import detect_frames as ds
import numpy as np

In [2]:
backcolor_set = (13,254,17)
threshold_for_backcolor = 10

In [3]:
corrected_error_file_path = '/home/damian/Downloads/에러수정/'
psd_num1 = '37_016.psd'
data_path = '/home/damian/test/'
psd_num2 = '037PSD/016/'

In [25]:
psd = PSDImage.load('/home/damian/test/037PSD/001.psd')

In [5]:
backcolor_in = False
frame_in = False
line_drawing_in = False
bottom_painting_in = False

backcolor_visible = True
frame_visible = True
line_drawing_visible = True
bottom_painting_visible = True

i = 0
for layer in psd.layers:
    if layer.name == 'backcolor':
        backcolor_idx = i
        backcolor_in = True
        if layer.visible == False:
            backcolor_visible = False
            
    if layer.name == 'frame':
        frame_idx = i
        frame_in = True
        if layer.visible == False:
            frame_visible = False
            
    if layer.name == 'line_drawing':
        line_drawing_idx = i
        line_drawing_in = True
        if layer.visible == False:
            line_drawing_visible = False
            
    if layer.name == 'bottom_painting':
        bottom_painting_idx = i
        bottom_painting_in = True
        if layer.visible == False:
            bottom_painting_visible = False
    i += 1

if backcolor_in == False:
    print ("backcolor doesn't exist")
    backcolor_idx = 100
if frame_in == False:
    print ("frame doesn't exist")
    frame_idx = 100
if line_drawing_in == False:
    print ("line_drawing doesn't exist")
    line_drawing_idx = 100
if bottom_painting_in == False:
    print ("bottom_painting doesn't exist")
    bottom_painting_idx = 100

if backcolor_visible == False:
    print ("backcolor is not visible")
if frame_visible == False:
    print ("frame is not visible")
if line_drawing_visible == False:
    print ("line_drawing is not visible")
if bottom_painting_visible == False:
    print ("bottom_painting is not visible")    

In [7]:
psd.layers = [psd.layers[frame_idx], psd.layers[backcolor_idx]]
backcolor_frame_img = psd.as_PIL_merged()
backcolor_frame_img = backcolor_frame_img.convert('RGB')
backcolor_frame_img.save(data_path + psd_num2 + 'backcolor_frame.png')
del psd

In [12]:
img = cv2.imread(data_path + psd_num2 + 'backcolor_frame.png')[:,:,::-1]
scenes = ds.detect_frames(img, data_path + psd_num2 + 'detected_backcolor_frame.png')

In [14]:
psd = PSDImage.load(corrected_error_file_path + psd_num1)
picture_img = psd.as_PIL()
picture_img.save(data_path + psd_num2 + 'picture.png')
backcolor_img = psd.layers[backcolor_idx].as_PIL()
psd.layers = [psd.layers[line_drawing_idx], psd.layers[bottom_painting_idx]]
lineDrawing_bottomPainting_img = psd.as_PIL_merged()
green_backcolor = Image.new('RGBA', backcolor_img.size, backcolor_set)
alpha_composite = Image.alpha_composite(green_backcolor, lineDrawing_bottomPainting_img)
RGB_alpha_composite = alpha_composite.convert('RGB')
RGB_alpha_composite.save(data_path + psd_num2 + 'user_backcolor.png')
del psd

In [ ]:
psd = PSDImage.load(corrected_error_file_path + psd_num1)
psd.layers = [psd.layers[line_drawing_idx], psd.layers[bottom_painting_idx], psd.layers[backcolor_idx]]
input_img = psd.as_PIL_merged()
input_img.save(data_path + psd_num2 + 'input_picture.png'
del psd               

In [23]:
i = 0
img1 = cv2.imread(data_path + psd_num2 + 'user_backcolor.png')
img2 = cv2.imread(data_path + psd_num2 + 'input_picture.png')

seq_array = np.lexsort((scenes[:,0], scenes[:,1]))
seq_list = seq_array.tolist()
scenes_list = scenes.tolist()
sorted_scenes_list = []

for idx in seq_list:
    sorted_scenes_list.append(scenes_list[idx])

for x, y, w, h in sorted_scenes_list:
    center_x = x + int(w/2)
    center_y = y + int(h/2)
    cropped1 = transform.crop(img1, (center_x,center_y), w, h)
    cropped2 = transform.crop(img2, (center_x,center_y), w, h)
    is_backcolor = (cropped1 == backcolor_set[::-1]).all(2)
    r, c = np.where(is_backcolor == False)

    if len(c) > threshold_for_backcolor:
        i+= 1
        cv2.imwrite(data_path + psd_num2 +'/output_cropped_img'+str(i)+'.png', cropped1)
        cv2.imwrite(data_path + psd_num2 +'/input_cropped_img'+str(i)+'.png', cropped2)

In [ ]:
with open(work_path+'/name_error_psd.txt','wb') as f :
    	pickle.dump(name_error_list, f)